In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-5a2b2804-d009-7029-6a76-48e0e270fb43)


In [4]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
print(f'Total training samples: {len(train_df)}')
print(f'Total test samples {len(test_df)}')
print(f'Total samples: {len(train_df) + len(test_df)}')

Total training samples: 7613
Total test samples 3263
Total samples: 10876


# Visualizing a text dataset

In [11]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Police Officer Wounded Suspect Dead After Exchanging Shots: Richmond police officer wounded suspect killed a... http://t.co/w0r8EAOKA0

---

Target: 0 (not real disaster)
Text:
Don't be the cause of your own self destruction

---

Target: 1 (real disaster)
Text:
#hot  Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/UMgD92wLjA #prebreak #best

---

Target: 1 (real disaster)
Text:
Cross-border terrorism: Pakistan caught red-handed again - The Times of India http://t.co/uiqsfgZoOx

---

Target: 1 (real disaster)
Text:
#News : The Latest: More Homes Razed by Northern California Wildfire - New York Times http://t.co/5kBRZZmf8c #TAFS #FB100%

---



# Split data

In [20]:

from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [21]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [22]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Converting text into numbers

## Text vectorization (tokenization)

In [23]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [25]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [26]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [27]:
text_vectorizer.adapt(train_sentences)

In [29]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [30]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
3 Former Executives to Be Prosecuted in Fukushima Nuclear #Disaster.

The story: http://t.co/7uFnxxaVqs via @nytimes      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 118,  952, 2120,    5,   21, 2014,    4,  951,  105,   75,    2,
         451,    1,   49, 2030]])>

In [31]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating an Embeddings

In [36]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                              output_dim = 128,
                              embeddings_initializer = 'uniform',
                              input_length = max_length,
                              name = 'embedding_1')

embedding

In [37]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
ANNIHILATED IN DAMASCUS: SYRIAN ARMY GRINDS Û÷ALLOOSH AND HIS GANG INTO THE MANURE PILE
http://t.co/7rakhP3bWm      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01568788, -0.02162093, -0.03943061, ...,  0.01801162,
         -0.00121324, -0.00703096],
        [-0.00137877, -0.03226204,  0.00332443, ..., -0.02137871,
          0.00667497, -0.02477685],
        [-0.04698575,  0.03873893, -0.01800288, ...,  0.04110261,
         -0.03871037, -0.01309335],
        ...,
        [-0.04698575,  0.03873893, -0.01800288, ...,  0.04110261,
         -0.03871037, -0.01309335],
        [-0.03727466,  0.04446247, -0.01240387, ..., -0.01257564,
         -0.00476855, -0.02696067],
        [-0.04698575,  0.03873893, -0.01800288, ...,  0.04110261,
         -0.03871037, -0.01309335]]], dtype=float32)>

In [41]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-1.56878829e-02, -2.16209292e-02, -3.94306071e-02, -3.42715755e-02,
       -8.65533203e-03,  3.78773101e-02,  3.84173654e-02, -3.17737237e-02,
       -2.65780929e-02,  8.65188986e-03, -2.33773347e-02, -3.13104987e-02,
        2.81218924e-02, -9.93556902e-03, -1.50739066e-02, -2.67108921e-02,
       -4.26465981e-02, -1.98302753e-02,  3.02794687e-02,  4.34178449e-02,
        4.29853909e-02,  9.85530764e-03,  1.04292035e-02, -2.86610480e-02,
       -1.63353309e-02,  4.16454338e-02, -2.81956084e-02, -2.07034107e-02,
        8.16876814e-03, -3.26364115e-03, -1.67608261e-03, -2.40557268e-03,
        1.50952004e-02, -4.53026183e-02, -2.80663259e-02, -2.04274300e-02,
       -2.75483485e-02, -1.57351010e-02,  4.42341305e-02,  3.79257686e-02,
       -3.22907344e-02,  3.61167267e-03, -2.71551255e-02,  1.58139206e-02,
       -4.09102216e-02, -1.96234584e-02,  1.18825920e-02,  1.26422532e-02,
        3.08724754e-02,  4.03400175e-02,  2.90347971

# Model 0: Naive Bayes

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [43]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [44]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [45]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = "weighted")
  model_results = {'accuracy': model_accuracy,
             'precision': model_precision,
             'recall': model_recall,
             'f1': model_f1}

  return model_results


In [46]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

# Model 1: A simple dense model

In [48]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs= layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name = 'model_1_dense')

model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [49]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [50]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 13s 35ms/step - loss: 0.6099 - accuracy: 0.6930 - val_loss: 0.5367 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4417 - accuracy: 0.8181 - val_loss: 0.4693 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3469 - accuracy: 0.8609 - val_loss: 0.4590 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2853 - accuracy: 0.8914 - val_loss: 0.4639 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2384 - accuracy: 0.9117 - val_loss: 0.4763 - val_accuracy: 0.7874


In [51]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.4763 - accuracy: 0.7874


[0.47634097933769226, 0.787401556968689]